In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
from urllib2 import urlopen
import urllib2
from bs4 import BeautifulSoup
import io
import pandas as pd
from io import StringIO, BytesIO
import pandas as pd
import numpy as np
import re
import json
import time
from tqdm import tqdm

In [1]:
!which python

/Users/istar/anaconda/bin/python


# Part 1

In [58]:
# table raws
class BasicRaw:
    def __init__(self):
        self.text = "None"
    def get_text(self):
        return self.text.encode("utf-8").strip()
class TitleRaw(BasicRaw):
    def __init__(self, title):
        self.text = title

class KeyValueRaw(BasicRaw):
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.text = key+" "+value
    def get_kvp(self):
        return self.key, self.value
class FunctionsAndOther(BasicRaw):
    def __init__(self, string, prev_header="None"):
        self.text = string
        self.prev_header = prev_header
class Description(BasicRaw):
    def __init__(self, text):
        self.text = text
    def __str__(self):
        return self.text
class Raw:
    # takes raw raw basically from tbody tag:) and that return one of TitleRow or KeyValueRaw object
    # prev is previous title
    def __init__(self, soup, prev="None"):
        self.soup = soup
        print self.soup.prettify()
        self.prev = prev
    def get_particular_raw_data(self):
        req = self.soup.find('span', id="watchNotes")
        if not(req is None):
            return Description(req.text)
        req = self.soup.find('h4')
        if not(req is None):
            # find a header raw
            return TitleRaw(req.text)
        req = self.soup.find_all('td')
        if len(req) == 2:
            # found key-value pair (we hope)
            return KeyValueRaw(req[0].text.strip(), req[1].text.strip())
        else:
            return FunctionsAndOther(','.join([i.text for i in req]), self.prev)

In [ ]:
class BasePage:
    def __init__(self, url, proxy_seed = 0, proxies=None):
        self.url = url
        if proxies is None:
            self.proxies = []
        else:
            self.proxies = proxies
        self.ip_usage = proxy_seed
        # get's beautiful soup of page
        self.set_soup()

In [59]:
class Page(BasePage):
    def __init__(self, url, proxy_seed = 0, proxies=None):
        """
            this class scrap data form web page that include all info about item
            proxies - array of proxy ips
            proxy_seed is variable that determine proxy_id - can be the counter of Page class usage
        """
        self.url = url
        if proxies is None:
            self.proxies = []
        else:
            self.proxies = proxies
        self.ip_usage = proxy_seed
        # get's beautiful soup of page
        self.set_soup()
        # all data form rows
        self.rows = []
    def get_page(self):
        # open new ip each time using counter ip_usage
        prx = self.ip_usage % (len(self.proxies) + 1)
        self.ip_usage = (self.ip_usage + 1) % (len(self.proxies) + 1)
        if (prx == 0):
            test_proxy = urllib2.ProxyHandler({})
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            return urllib2.urlopen(self.url)
        else:
            prx -= 1
            test_proxy = urllib2.ProxyHandler(self.proxies[prx])
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            answer = urllib2.urlopen(self.url)
            return answer
    def set_soup(self):
        page = self.get_page()
        soup = BeautifulSoup(page, 'html.parser')
        self.soup = soup
    def get_table(self):
        # put all data from tables in array of raws 
        # where each raw is a particular object selected in Raw class
        data = []
        for table in self.soup.find_all('tbody'):
            # this is title object /is a inner rows-sub-box title/
            prev = "None"
            for raw in table.find_all('tr'):
                r = Raw(raw, prev)
                partic_obj = r.get_particular_raw_data()
                if isinstance(partic_obj, TitleRaw):
                    prev = partic_obj
                data.append(partic_obj)
        return data
    def get_js_data(self):
        data = self.soup.find_all('script')
        var_names = ["watchId", "merchantId", "watchName", "originalPath", "uuid", "merchantPage"]
        data_dict = {}
        for var_name in var_names:
            for i in data:
                values = re.findall(r'var {} = "(.*?)";'.format(var_name), str(i), re.DOTALL | re.MULTILINE)
                if len(values) != 0:
                    data_dict[var_name] = values[0]
        return data_dict
    def get_from_table(self):
        # get pandas object containing all data from raws
        raws = self.get_table()
        data = self.get_js_data()
        for raw in raws:
            if isinstance(raw, KeyValueRaw):
                r_kvp = raw.get_kvp()
                if r_kvp[0] == u'\xa0' or r_kvp[0] == u'\u20ac':
                    r_kvp = ["_OTHER_INFO_", r_kvp[1]]
                if r_kvp[0] in data:
                    # if keys collided
                    continue
                data[r_kvp[0]] = r_kvp[1]
            elif isinstance(raw, Description):
                data["_Description_"] = raw.get_text()
            elif isinstance(raw, FunctionsAndOther):
                if "_Functions_and_other" in data:
                    data["_Functions_and_other"] += '; ' + raw.get_text()
                else:
                    data["_Functions_and_other"] = raw.get_text()
        return data
    

In [60]:
test_item_url = "http://www.chrono24.com/chronoswiss/ch-7523-lunar-chronograph-ss-25186--id6294206.htm"

In [61]:
page = Page(test_item_url)

In [62]:
page.get_table()
print 1

In [63]:
page.get_from_table()

{u'': u'With box',
 u'Availability': u'Available now',
 u'Brand': u'Chronoswiss',
 u'Case diameter': u'38 mm',
 u'Case material': u'Steel',
 u'Clasp': u'Fold clasp',
 u'Clasp material': u'Steel',
 u'Code': u'25186',
 u'Condition': u'1 (mint)',
 u'Dial': u'Silver',
 u'Glass': u'Sapphire Glass',
 u'Location': u'United States, Massachusetts, Boston',
 u'Movement': u'Automatic',
 u'Price': u'$3,900',
 u'Ref. No.': u'7523',
 u'Thickness': u'14.5 mm',
 '_Description_': "Chronoswiss CH 7523 Lunar Chronograph, CH 7523 L, CH7523L, stainless steel on a strap with a stainless steel deployant buckle, automatic, date, chronograph, moon phases, small second indicator at 9 o'clock, silver gullioche dial, diameter: 38 mm, thickness: 14.5 mm, display back, sapphire crystal, water resistant, Like New with original box and blank warranty certificate.\n\n\n     $3,900",
 '_Functions_and_other': 'Moonphase, Chronograph, Date; Display Back, Small Seconds',
 'merchantId': '2092',
 'originalPath': '/chronoswi

In [64]:
page.get_js_data()

{'merchantId': '2092',
 'originalPath': '/chronoswiss/ch-7523-lunar-chronograph-ss-25186--id6294206.htm',
 'uuid': 'b7b07526-f314-4ad1-a73a-e824d1c5ea18',
 'watchId': '6294206',
 'watchName': 'Chronoswiss CH 7523 Lunar Chronograph SS (25186)'}

# Part 2

In [ ]:
class ModelPage:
    def __init__(self, page_url):
        self.page_url = url

In [65]:
mainData = dict()
sz = 0

In [66]:
main_page = "http://www.chrono24.com/chronoswiss"
adder = "/index.htm"
# pages ase ..index, index-2, index-3, index-4

In [67]:
txt = urllib2.urlopen(main_page + adder).read()

In [68]:
data = BeautifulSoup(txt, 'html.parser')

In [69]:
hrefs = data.find_all("div", class_="item")

In [70]:
urls = []
for h in hrefs:
    url_add = h.find("a")["href"]
    urls.append(main_page + url_add)

In [71]:
for i in tqdm(urls):
    sz += 1
    page_ = Page(i)
    dct = page_.get_from_table()
    for k in dct.keys():
        if k in mainData:
            mainData[k].append(dct[k])
        else:
            mainData[k] = ["None"] * (sz - 1)
            mainData[k].append(dct[k])
    for k in mainData:
        if not k in dct:
            mainData[k].append("None")
            

In [72]:
print mainData.keys()

In [73]:
data_to_csv = pd.DataFrame(mainData)

In [74]:
data_to_csv['Price']

0                 $4,250 [Negotiable]
1                              $2,745
2                              $6,500
3                             $17,995
4                 $8,495 [Negotiable]
5                              $7,950
6                              $3,900
7                 $3,600 [Negotiable]
8     3,500 € (= $3,954) [Negotiable]
9                              $6,250
10                $1,645 [Negotiable]
11                             $2,400
12                             $4,239
13                $1,995 [Negotiable]
14                             $3,450
15                $3,400 [Negotiable]
16                $4,100 [Negotiable]
17                            $17,500
18                            $22,500
19                $1,645 [Negotiable]
20                             $6,500
21                             $2,319
22                $2,995 [Negotiable]
23                             $3,350
24                            $11,450
25                             $9,590
26          

In [75]:
data_to_csv.to_csv("./Promo.csv", sep=";")